In [111]:
import numpy as np
import gym
from collections import deque
import pyprind
from time import time

In [112]:
q = deque()

In [114]:
q.append(9)

In [119]:
np.array([list(q)]).shape

(1, 1)

In [26]:
P_a0 = np.array([
        [0.5, 0.5],
        [0.0, 1.0]
    ])
P_a1 = np.array([
        [0.0, 1.0],
        [0.0, 1.0]
    ])
P_a2 = np.array([
        [1.0, 0.0],
        [0.0, 1.0]
    ])
P_all = np.concatenate([P_a0, P_a1, P_a2], axis=0)
P_list = [P_a0, P_a1, P_a2]

In [27]:
P_all.shape

(6, 2)

In [ ]:
R_all = np.array([
        [5.0, 10.0, -1000],
        [-1000, -1000, -1.0]
    ])

In [ ]:
gamma = 0.95
tol = 1e-10

In [ ]:
V = np.zeros(2)

In [ ]:
gamma*np.dot(P_all, V).reshape(R_all.shape, order='F')

In [ ]:
start = time()
for i in range(1000):
    V_old = np.copy(V)
    V = np.max(R_all + gamma*np.dot(P_all, V).reshape(R_all.shape, order='F'), axis=1)
    error = np.linalg.norm(V-V_old, 2)
    if error < tol:
        print('Converged after {:} iterations with error {:}'.format(i+1, error))
        break
print('Time taken:', time() - start, 's')
V


In [ ]:
Q = -1000*np.ones((2,3))
A = [[0, 1], [2]]
start = time()
for i in range(1000):
    for s in range(2):
        for a in A[s]:
            res = R_all[s, a]
            for sprime in range(2):
                #print(res)
                res += gamma*P_list[a][s, sprime]*np.max(Q[sprime])
            Q[s,a] = res
V = np.max(Q, axis=1)
print('Time taken:', time() - start, 's')
V

In [ ]:
Q

In [ ]:
V_pi = np.zeros(2)
pi = np.zeros(2, dtype=int)
for i in range(900000):
    pi_old = np.copy(pi)
    P_pi = np.array([P_list[pi[0]][0],P_list[pi[1]][1]])
    R_pi = np.array([R_all[0, pi[0]], R_all[1, pi[1]]])
#     V_pi = np.linalg.solve(a=np.eye(len(P_pi))-gamma*P_pi, b=R_pi) #Policy Iteration
    V_pi = R_pi + gamma*np.dot(P_pi, V_pi) #Modified Policy Iteration
    pi = np.argmax(R_all + gamma*np.dot(P_all, V_pi).reshape(R_all.shape, order='F'), axis=1)
    if np.allclose(pi, pi_old) and i > 10000:
        print('Converged after {:} iterations'.format(i+1))
        break
        pass
pi

In [ ]:
V_pi

In [ ]:
np.array([P_list[pi[0]][0],P_list[pi[1]][1]])

In [3]:
env = gym.make("FrozenLake-v0")
#env = gym.make("Taxi-v2")

In [ ]:
N = 10000
Q = np.zeros((env.env.nS, env.env.nA))
gamma = 0.1
tol = 1e-6
bar = pyprind.ProgBar(N)
for i in range(N):
    bar.update()
    Q_old = np.copy(Q)
    for s in range(env.env.nS):
        for a in list(env.env.P[s].keys()):
            res = 0.0
            for j, sprime in enumerate([k[1] for k in env.env.P[s][a]]):
                res += env.env.P[s][a][j][0]*(env.env.P[s][a][j][2] + gamma*np.max(Q[sprime]))
            Q[s,a] = res
    error = np.linalg.norm(Q-Q_old,2)
    if error < tol:
        print('Converged after {:} iterations and error is {:}'.format(i+1,error))
        break
print(bar)
V = np.max(Q, axis=1)

In [ ]:
env.env.P[5].keys()

In [ ]:
env.env.nS

In [ ]:
env.env.nS

In [4]:
V_policy = np.zeros(env.env.nS)
policy = np.zeros(env.env.nS, dtype=int)
 

In [5]:
env.env.P[10]

{0: [(0.3333333333333333, 6, 0.0, False),
  (0.3333333333333333, 9, 0.0, False),
  (0.3333333333333333, 14, 0.0, False)],
 1: [(0.3333333333333333, 9, 0.0, False),
  (0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 11, 0.0, True)],
 2: [(0.3333333333333333, 14, 0.0, False),
  (0.3333333333333333, 11, 0.0, True),
  (0.3333333333333333, 6, 0.0, False)],
 3: [(0.3333333333333333, 11, 0.0, True),
  (0.3333333333333333, 6, 0.0, False),
  (0.3333333333333333, 9, 0.0, False)]}

In [20]:

nA, nS = env.env.nA, env.env.nS


In [21]:
T = np.zeros([nS, nA, nS])
R = np.zeros([nS, nA, nS])

In [41]:
for s in range(nS):
    for a in range(nA):
        transitions = env.env.P[s][a]
        for p_trans,next_s,rew,done in transitions:
#             print(next_s)
            T[s,a,next_s] += p_trans
            
            R[s,a,next_s] = rew
        T[s,a,:]/=np.sum(T[s,a,:])

In [42]:
# for s in range(nS):
#      for a in range(nA):
print(env.env.P[10][2])

[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 11, 0.0, True), (0.3333333333333333, 6, 0.0, False)]


In [150]:
P_list = [np.zeros([nS,nS]) for _ in range(nA)]
R_list = [-1000*np.ones((nS, nS)) for _ in range(nA)]
for s in range(nS):
    for a in range(nA):
        transitions = env.env.P[s][a]
        for p_trans,next_s,rew,done in transitions:
            P_list[a][s,next_s]= p_trans
            R_list[a][s,next_s] = rew
       

In [151]:
R_all = np.vstack([np.sum(R_list[i]*P_list[i], axis=1) for i in range(nA)]).T

In [152]:
R_all

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.33333333,  0.33333333,  0.33333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [120]:
T

array([[[ 0.33333333,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.33333333,  0.33333333,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.33333333,  0.33333333,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.33333333,  0.33333333,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.33333333,  0.33333333,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.33333333,  0.        ,  0.33333333, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.33333333,  0.33333333, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.33333333,  0.33333333,  0.33333333, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.33333333,  0.33333333, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.33333333,  0.        , ...,  0.        ,
          0.